# f1_extract_sentences_with_entities
From the file sentences.txt extract all the sentence who contain entities and write them into .txt file

In [71]:
import urllib.request
import re
from bs4 import BeautifulSoup
from dateutil.parser import parse
import csv
import pandas as pd
import requests
import nltk
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.tag import StanfordNERTagger
from nltk.tokenize import MWETokenizer
import import_ipynb
import c1_extract_paragraphe_issue as c1 
import c2_extract_sentence_issue as c2 
import collections
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tatianacogne/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tatianacogne/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [73]:
def write_list_collected(sentences,s):
    """Write list_sentence_with_country.txt file. """
    outF = open(s, "w")
    for line in sentences :
        # write line to output file
        outF.write(line)
        outF.write('\n')
    outF.close()

In [74]:
def write_list_collected_issue(list_with_two_entities, number):
    """Write list_sentence_with_country.txt file. """
    
    s = "s2e"+str(number)
    outF = open(s+".txt", "w")
    for line in list_with_two_entities :
        # write line to output file
        outF.write(line)
        outF.write('\n')
    outF.close()

In [75]:
def extract_from_txt_sentences(txt_file):
    """ Open file that contains all the sentences."""
    list_tp = open(txt_file)
    return list_tp

In [76]:
def clean_tp(sentence):
    """ Clean the sentence by removing special char."""
    s = sentence.replace("\r\n\s\s+"," ")
    s = s.replace("\r\n"," ")
    s = s.replace("\s\s+"," ")
    s = s.replace("\\."," ")
    s = s.replace("\\r\\n"," ")
    p = re.compile(r'<.*?>')
    return p.sub('', s)

In [77]:
def tokenize_sentence(sentence):
    """SPlit the sentence in a way that the entities are together and will be able to be detected."""
    # Extract list entities
    list_entities = [s.replace('\n','') for s in list(open('entities_clean.txt'))]
    list_entities = [s.replace('class="textstory"','') for s in list_entities]
    
    tokens_entities = [l.split(' ') for l in list_entities]

    tokenizer1 = MWETokenizer(tokens_entities, separator=' ')
    tokenizer2 = MWETokenizer([['G-77','CHINA']], separator='/')
    tokenizer3 = MWETokenizer([['G-77/',' CHINA']], separator=' ')

    line = sentence.replace(",","")
    line_splited = word_tokenize(line)
    tokens = tokenizer1.tokenize(line_splited) 
    tokens = tokenizer2.tokenize(tokens) 
    tokens = tokenizer3.tokenize(tokens) 
    tokens = [clean_tp(token) for token in tokens]

    return tokens

In [78]:
def extract_s2e_issue():
    """ Extract all the sentence with at least one party inside. """

    #List sentences
    #paragraphes = c1.extract_paragraphes_from_issue(number)
    sentences = c2.extract_from_txt_sentences(open("sentences_s1e.txt"))
    print(len(sentences))
    sentences = list(set(sentences))
    print(len(sentences))
    list_entities = [s.replace('\n','') for s in list(open('entities_clean.txt'))]
    #Create list that wil contain all the sentences with at least two entities
    sentences_s2e = []
    for i, s in enumerate(sentences):
    #Split line into words with tokenizer to detetc entity
    
        tokens = tokenize_sentence(s)
        done = False
        for e1 in list_entities: 
            for e2 in list_entities:
                if(set([e1,e2]).issubset(set(tokens)) and e1 != e2):
                    sentences_s2e.append(s)
                    done = True
                    break
            if done:
                break
        print(f'{(i+1)/len(sentences)*100:.2f}%', end='\r')
    s = "sentences_s2e.txt"
    write_list_collected(sentences_s2e,s)
    return sentences_s2e

In [80]:
def extract_s1e_issue():
    """ Extract all the sentence with at least one party inside. """

    #List sentences
    #paragraphes = c1.extract_paragraphes_from_issue(number)
    sentences = c2.extract_from_txt_sentences(open("sentences.txt"))
    sentences = list(set(sentences))
    list_entities = [s.replace('\n','') for s in list(open('entities_clean.txt'))]
    #Create list that wil contain all the sentences with at least two entities
    sentences_s1e = []
    for i, s in enumerate(sentences):
    #Split line into words with tokenizer to detect entity
        tokens = tokenize_sentence(s)
        done = False
        for e1 in list_entities: 
            if(set([e1]).issubset(set(tokens)) ):
                sentences_s1e.append(s)
                done = True
                break

        print(f'{(i+1)/len(sentences)*100:.2f}%', end='\r')
    s = "sentences_s1e.txt"
    write_list_collected(sentences_s1e,s)
    return sentences_s1e

In [47]:
def find_pos_tagged_s2e(list_s2e):
    """ Use NLTK to pos_tag all the sentences from list_s2e and return a list of all the sentences pos_tagged"""
    verbs = []
    nnp = []
    pos_tagged = []
    for s in list_s2e:
        tokens = tokenize_sentence(s)
        pos_tagged += nltk.pos_tag(tokens)   
   
    return pos_tagged

In [72]:
def find_verbs_s2e(sentences_s2e,number_most_c):
    """ Find all the verbs VBD and return the number_most_c of them into a list"""
    pos_tagged = find_pos_tagged_s2e(sentences_s2e)
    verbs = filter(lambda x:x[1]=='VBD',pos_tagged)
    verbs = list(verbs)
    counter=collections.Counter(verbs)
    return counter.most_common(number_most_c)


In [81]:
sentences_s1e = extract_s1e_issue()

100.00%

NameError: name 'list_with_two_entities' is not defined

In [29]:
sentences_s2e = extract_s2e_issue()

21068
21068
100.00%100.00%